In [1]:
# Importamos las librerías.

import time
import requests
import pandas as pd
from tqdm import tqdm  # Importar tqdm para las barras de progreso.
import requests
from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select
import time  
from time import sleep
import csv 
pd.set_option('display.max_columns', None)

## Extracción API

In [ ]:
def peliculas_por_genero(generos_lista, rango_anios):
    # Lista vacía para almacenar las películas.
    lista_peliculas = []
    
    # Iteramos por cada género con barra de progreso.
    for genero in tqdm(generos_lista, desc="Procesando géneros"):
        # Iteramos por cada año en el rango con barra de progreso.
        for year in tqdm(rango_anios, desc=f"Año {genero}", leave=False):
            url_genero = f"https://dev.adalab.es/api/cinema/movies?year={year}&genre={genero}"
            
            try:
                # Llamada a la API.
                llamada = requests.get(url_genero)
                # Pausa para evitar sobrecargar la API.
                time.sleep(3)

                # Verificamos que la llamada fue exitosa (código 200).
                if llamada.status_code == 200:
                    # Convertimos la respuesta a formato JSON.
                    datos_peliculas = llamada.json()

                    # Iteramos por película para obtener los datos relevantes.
                    for pelicula in datos_peliculas.get('results', []):
                        titulo = pelicula.get("title")
                        identificacion = pelicula.get('idOwner')
                        tipo = pelicula.get('type')
                        anio = pelicula.get('year')
                        genero_pelicula = pelicula.get('genre')

                        # Agregamos los datos de la película a la lista.
                        pelicula_tupla = (titulo, identificacion, tipo, anio, genero_pelicula)
                        lista_peliculas.append(pelicula_tupla)
                else:
                    print(f"Error {llamada.status_code}: No se pudo obtener datos para el año {year} y género {genero}")

            except Exception as e:
                # Captura cualquier error que ocurra durante la llamada o procesamiento.
                print(f"Error al procesar el año {year} y género {genero}: {str(e)}")
    
    # Retornamos la lista de películas.
    return lista_peliculas

# Lista de géneros.
generos_lista = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]

# Rango de años.
rango_anios = range(2000, 2016)

# Obtener la lista de películas por género y año.
lista_peliculas = peliculas_por_genero(generos_lista, rango_anios)

# Convertir a DataFrame.
df_peliculas = pd.DataFrame(lista_peliculas, columns=['Titulo', 'ID', 'Tipo', 'Año', 'Genero'])

# Guardar el DataFrame en un archivo CSV.
df_peliculas.to_csv("peliculas_generos.csv", index=False)

print("Extracción completa. Los datos han sido guardados en 'peliculas_generos.csv'.")

Utilización de csv para Selenium

In [10]:
# Open the CSV file

with open('peliculas_generos.csv', mode='r', newline='') as file:
    # Create a CSV reader object
    doc_csv = csv.reader(file)
    # Read and print each row in the CSV file
    for row in doc_csv:

        print(row)

['Titulo', 'ID', 'Tipo', 'Año', 'Genero']
['Istota', 'tt0102362', 'Movie', '2000', 'Drama']
['Gensomaden Saiyuki', 'tt0111056', 'TV Series', '2000', 'Drama']
['For the Cause', 'tt0113092', 'Movie', '2000', 'Drama']
['Florentino y el diablo', 'tt0113086', 'Movie', '2000', 'Drama']
['Karobaar: The Business of Love', 'tt0116748', 'Movie', '2000', 'Drama']
['Chateaubriand - Cabeça de Paraíba', 'tt0115874', 'Short', '2000', 'Drama']
['Gang', 'tt0116391', 'Movie', '2000', 'Drama']
['Consequence', 'tt0115937', 'Movie', '2000', 'Drama']
['Fa yeung nin wah', 'tt0118694', 'Movie', '2000', 'Drama']
['Dumped', 'tt0119027', 'Video', '2000', 'Drama']
['November 1828', 'tt0079644', 'Movie', '2001', 'Drama']
['The Killing Seasons', 'tt0113545', 'Short', '2001', 'Drama']
['Albela', 'tt0118578', 'Movie', '2001', 'Drama']
["Don's Plum", 'tt0119004', 'Movie', '2001', 'Drama']
['Glitter', 'tt0118589', 'Movie', '2001', 'Drama']
['Camarate', 'tt0120624', 'Movie', '2001', 'Drama']
['Berlin Diaries, 1940-45', 

Limpieza registros tipo Movie

In [11]:

# Leer el archivo CSV y extraer solo los registros "Movie"
df = pd.read_csv('/Users/andreagonzalez/Documents/Adalab/Proyectos_Andrea/Proyecto_2/peliculas_generos.csv',index_col = 0)

# Filtrar los registros donde el tipo sea 'Movie'.
df_movies = df[df['Tipo'] == 'Movie']

print(len(df_movies))


611


In [12]:
df_movies 

,ID,Tipo,Año,Genero
Titulo,,,,
Istota,tt0102362,Movie,2000,Drama
For the Cause,tt0113092,Movie,2000,Drama
Florentino y el diablo,tt0113086,Movie,2000,Drama
Karobaar: The Business of Love,tt0116748,Movie,2000,Drama
Gang,tt0116391,Movie,2000,Drama
...,...,...,...,...
Dark Blood,tt0293069,Movie,2012,Thriller
The Yellow Wallpaper,tt0790788,Movie,2012,Thriller
The Grind,tt0906013,Movie,2012,Thriller


## Extracción info películas IMBD

In [ ]:
# Iniciar el navegador.
driver = webdriver.Chrome()
print("Abriendo Google.📩")

time.sleep(5)
# Maximizar la ventana del navegador.
driver.maximize_window()
print("Pantalla maximizada.💻")
time.sleep(3)

# Abrir página web de IMDb.
driver.get("https://www.imdb.com/")
print("Accediendo a la web.🔄")
time.sleep(3)

# Aceptar cookies.
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
print("Cookies aceptadas.🍪")
time.sleep(5)

datos_imbd = []

# Iterar sobre un rango específico de filas (0 a 359) del DataFrame.
for index, row in df_IMDB.iloc[360:720].iterrows():
    # Buscar la película por su id en IMDb.
    url_pelicula = f"https://www.imdb.com/title/{row['ID']}/"
    driver.get(url_pelicula)
    time.sleep(5)

    try:
        # Extraer puntuación.
        try:
            puntuacion = driver.find_element("css selector", "span.sc-eb51e184-1.ljxVSS").text
        except Exception as e:
            puntuacion = "Desconocido"
        print(f"Puntuación: {puntuacion}")

        # Extraer dirección.
        try:
            direccion = driver.find_element("css selector", "div.sc-491663c0-10.iaQXlA ul:nth-child(1)").text
        except Exception as e:
            direccion = "Desconocido"
        print(f"Dirección: {direccion}")

        # Extraer guionista.
        try:
            guionista = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div').text
        except Exception as e:
            guionista = "Desconocido"
        print(f"Guionista: {guionista}")

        # Extraer sinopsis.
        try:
            argumento = driver.find_element("css selector", "span.sc-2d37a7c7-2.ggeRnl").text
        except Exception as e:
            argumento = "Desconocido"
        print(f"Argumento: {argumento}")

        # Extraer nombre de la película y género
        nombre_pelicula = index  # Esto toma el índice del DataFrame como el nombre de la película
        genero = row['Genero']
        print(f"Nombre: {nombre_pelicula}")
        print(f"Genero: {genero}")

        # Guardar la tupla con los datos.
        datos_imdb.append((puntuacion, direccion, guionista, argumento, nombre_pelicula, row['ID'], genero))

    except Exception as e:
        print(f"Error al procesar la película con ID {row['ID']}: {e}")
        continue

# Cerrar el navegador.
driver.quit()

In [ ]:
# Ver los datos extraídos.
for pelicula in datos_imdb:
    print(pelicula)

In [ ]:
# Convertimos los archivos en Data Frame
df_pelis = pd.DataFrame(datos_imdb)

df_pelis

In [ ]:
# Guardar los datos en un archivo CSV.
with open('peliculas_imdb1.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Escribir el encabezado.
    writer.writerow(['Puntuación', 'Dirección', 'Guionista', 'Argumento', 'Nombre de la película', 'ID de la película', 'Genero'])
    
    # Escribir los datos.
    writer.writerows(datos_imdb)

print("Datos guardados en peliculas_imdb.csv")

## Extracción información actores IMDB

In [5]:

# Leer el archivo CSV y extraer solo la columna 'id'.
df_IMDB = pd.read_csv('/Users/andreagonzalez/Documents/Adalab/Proyectos_Andrea/Proyecto_2/peliculas_imdb1.csv',index_col = 0)

# Filtrar los registros donde el tipo sea 'Movie'.

df_IMDB

,Dirección,Guionista,Argumento,Nombre de la película,ID de la película,Genero
Puntuación,,,,,,
6.3,DirectorAndrzej CzarneckiWriterMikhail Shelekh...,Mikhail Shelekhov,NaN,Istota,tt0102362,Drama
3.4,DirectorsDavid DouglasTim DouglasWritersDavid ...,David DouglasChristopher Salazar,Earth is in a state of constant war and two co...,For the Cause,tt0113092,Drama
7.0,Desconocido,Desconocido,"Florentino is a young horseman and singer, a f...",Florentino y el diablo,tt0113086,Drama
3.7,DirectorRakesh RoshanWritersSachin BhowmickRav...,Sachin BhowmickRavi KapoorSagar Sarhadi,Wealthy Rajiv Sinha and middle-classed Amar Sa...,Karobaar: The Business of Love,tt0116748,Drama
6.3,DirectorMazhar KhanStarsJackie ShroffNana Pate...,Jackie ShroffNana PatekarKumar Gaurav,"After falling prey to underworld, four friends...",Gang,tt0116391,Drama
...,...,...,...,...,...,...
6.9,Desconocido,Ted ElliottTerry RossioKarey Kirkpatrick,Two swindlers get their hands on a map to the ...,The Road to El Dorado,tt0138749,Fantasy
4.6,Desconocido,Gregory WidenEric BerntGillian Horvath,Immortals Connor and Duncan MacLeod must join ...,Highlander: Endgame,tt0144964,Fantasy
6.5,DirectorAlexandra KingWriterAlexandra KingStar...,Alexandra King,Two draq queens go on a Bonnie & Clyde rampage...,Red Lipstick,tt0144574,Fantasy


In [ ]:
# Iniciar el navegador.
driver = webdriver.Safari()
print("Abriendo Google.📩")

time.sleep(5)
# Maximizar la ventana del navegador.
driver.maximize_window()
print("Pantalla maximizada.💻")
time.sleep(3)

# Abrir página web de IMDb.
driver.get("https://www.imdb.com/")
print("Accediendo a la web.🔄")
time.sleep(3)

# Aceptar cookies.
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
print("Cookies aceptadas.🍪")
time.sleep(5)


datos_actores_imdb = []
actores_url= []

# Iterar sobre un rango específico de filas 0-310 del DataFrame.
for index, row in df_IMDB.iloc[0:311].iterrows():

    # Buscar la película por su id en IMDb.
    url_pelicula = f"https://www.imdb.com/title/{row['ID de la película']}/"
    driver.get(url_pelicula)
    time.sleep(5)

    try:
        elemento = driver.find_element("css selector",'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid')
        print('Explorando los actores')
    except Exception as e:
            print(f"Error al procesar la película con ID {row['ID de la película']}: {e}")
            
    time.sleep(5)

    for i in range(1,11):
        
         # Sacamos la información relevante de cada actor/actriz.
            
        try:
            act_imbd = elemento.find_element("css selector", f"#_next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-griditem.ipc-page-griditem--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.dGyVLT.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-castgrid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler_grid > div:nth-child({i}) > div.sc-bfec09a1-7.kSFMrr > a").text
            # Nombre actor/actriz. 
            nombre_actor= act_imbd.text
            print(f"Nombre: {nombre_actor}")
            actores_url= act_imbd.get_attribute('href')
                   
        except:
            print ('No se encontro la url')
                
        
        sleep(7)


        for actor in actores_url:
            url_actor = driver.get(actor)
            print('Acceso a pagina del actor')

                # Año nacimiento.
            try:
                nacimiento_imbd = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(22) > div.sc-f65f65be-0.dQVJPm > ul > li:nth-child(4) > div > ul > li.ipc-inline-list__item.test-class-react > a").text
                print (f'Se ha agregado la fecha de nacimiento{nacimiento_imbd}')
            except:
                nacimiento_imbd = "Desconocido"    
                print(f"Año nacimiento: {nacimiento_imbd}")    

                sleep(7)

                # ¿Por qué es conocid@?
                try:
                    for x in range(1,5):
                        obras_imbd = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-1c5b3162-0.dsjiSY > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({x})").text
                        print('Las obras {obras_imbd}')
                        
                except:
                    obras_imbd = "Desconocido"            
                    print(f"Obras: {obras_imbd}")    

                sleep(5)

                # ¿Qué hace?
                try:
                    roles_imbd = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text
                    print('Roles {roles_imbd}')
                    
                except Exception as e:
                    roles_imbd = "Desconocido"
                    print(f"Roles: {roles_imbd}")    

                sleep(5)

                # Premios.
                try:
                    premios_imbd = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > div > section > div > div.sc-4b498b6e-1.jDpRYv.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div > ul > li > div > ul > li > span").text
                    print('Premios {premios_imbd}')
                except:
                    premios_imbd = "Desconocido"
                    print(f"Premios: {premios_imbd}")


                # Extraer nombre de la película y género.
                nombre_pelicula = index  # Esto toma el índice del DataFrame como el nombre de la película.
                genero = row['Genero']
                print(f"Nombre: {nombre_pelicula}")
                print(f"Genero: {genero}")         

                # Guardar la tupla con los datos.
                datos_actores_imdb.append((act_imbd, nacimiento_imbd, obras_imbd, roles_imbd, premios_imbd, nombre_pelicula, row['ID de la película'], genero))
            
    

# Cerramos el navegador,
driver.close()